In [1]:
from keras.models import model_from_json
from functools import partial
from itertools import islice
import numpy as np
import sys
import json

Using TensorFlow backend.


In [2]:
model_name = 'input64x95_layers8_params5492k_h7b54'
model_filename = 'models/%s.model.json' % (model_name, )
with open(model_filename) as f:
    model = model_from_json(f.read())
model.name = model_name

model.summary()
print('Input shape: ' + repr(model.input_shape))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_1 (LSTM)                    (None, 64, 512)       1245184     lstm_input_1[0][0]               
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 64, 512)       0           lstm_1[0][0]                     
____________________________________________________________________________________________________
lstm_2 (LSTM)                    (None, 64, 512)       2099200     dropout_1[0][0]                  
____________________________________________________________________________________________________
dropout_2 (Dropout)              (None, 64, 512)       0           lstm_2[0][0]                     
___________________________________________________________________________________________

In [3]:
with open('models/' + model.name + '.chars.json') as f: 
    indices_char = {int(i): c for (i, c) in json.load(f).items()}
    char_indices = {c: i for (i, c) in indices_char.items()}

In [4]:
weights_filename = 'models/input64x95_layers8_params5492k_h7b54.weights_loss1.2726.h5'
print('Loading weights...')
model.load_weights(weights_filename)

Loading weights...


In [5]:
def char2vec(c):
    ci = char_indices[c]
    return np.array([int(i == ci) for i in range(len(char_indices))])

def vec2char(v, diversity = 0, random_state = np.random):
    v = np.ma.filled(np.exp(np.ma.log(v) / np.exp(diversity)), 0)
    v = v / sum(v)
    return indices_char[random_state.choice(len(indices_char), p=v)]

In [6]:
def speak(seed="", random_state = np.random, diversity=0):
    input_shape = model.input_shape[1:]
    X = random_state.rand(*input_shape)
    X = np.ones(input_shape)

    for i in range(max(input_shape[0] - len(seed), 0), input_shape[0]):
        X[i] = char2vec(seed[len(seed) - input_shape[0] + i])

    v2c = partial(vec2char, random_state=random_state)
    
    while True:
        y = model.predict(np.expand_dims(X, axis=0))[0]
        X = np.roll(X, shift=-1, axis=0)
        char = v2c(y, diversity=diversity)
        X[-1] = char2vec(char)
        yield char
        

def get_sentence(gen):
    for c in gen:
        yield c
        if c in ['?', '!', '.']: break



In [14]:
seed = "Sky is green. "
rnd = np.random.RandomState(1)
        
yudkowsky = speak(seed=seed, random_state=rnd, diversity=-0.2)

for c in get_sentence(islice(yudkowsky, 1000)):
    sys.stdout.write(c)
    sys.stdout.flush()
print()

The same rule products from extravology, which never even way to be an answer a follower positive free billion things of moral unlooks?
